In [1]:
import pandas as pd
import numpy as np
import json 

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import contextily as cx
from shapely.geometry import Point

import sklearn.cluster as cluster
import hdbscan 

import gc

In [2]:
# All the data cleaning
df = pd.read_csv("../CSV/Liquor_Licenses.csv")
recode = ['BREW_PUB', 'ENTERTAINMENT',
       'SALES_CONSUMPTION', 'SIDEWALK_CAFE', 'SUMMER_GARDEN', 'TASTING',
       'WINE_PUB', 'COVERCHARGE', 'DANCING',
       'OFFPREMISESTORAGE', 'STORAGEFACILITY', 'DISTILLERY_PUB','GAMES_OF_SKILL',
          'SPORTS_WAGGERING']
df[recode] = df[recode].replace({np.nan:0,'CHECKED':1 })
#Replace TYPE
df.TYPE = df.TYPE.replace({'Retail-Class B': 'Retail - Class B',
                'Retail-Liquor Store':'Retail - Liquor Store',
                 'Retail-Full Service Grocery':'Retail - Full Service Grocery'})
dup_list = df[df.duplicated(subset='LICENSE')].LICENSE.to_list()
dup = df[df.LICENSE.isin(dup_list)]
# Drop duplicates for License and address
df = df.drop_duplicates(subset = ['LICENSE','ADDRESS'], keep = 'last')
# Drop once again to get rid of the row with wrong address for Gallaudet
df = df.drop_duplicates(subset = ['LICENSE'], keep = 'first')

In [3]:
df.shape

(2123, 32)

In [4]:
df.columns

Index(['OBJECTID', 'LICENSE', 'TRADE_NAME', 'APPLICANT', 'CLASS', 'ADDRESS',
       'ADDRID', 'X', 'Y', 'STATUS', 'TYPE', 'BREW_PUB', 'ENTERTAINMENT',
       'SALES_CONSUMPTION', 'SIDEWALK_CAFE', 'SUMMER_GARDEN', 'TASTING',
       'WINE_PUB', 'WARD', 'ZIPCODE', 'SMD', 'ANC', 'COVERCHARGE', 'DANCING',
       'OFFPREMISESTORAGE', 'STORAGEFACILITY', 'DISTILLERY_PUB', 'LONGITUDE',
       'LATITUDE', 'TOTAL_CAPACITY', 'GAMES_OF_SKILL', 'SPORTS_WAGGERING'],
      dtype='object')

In [5]:
df[["LATITUDE","LONGITUDE","WARD"]].sample(20).to_numpy()

array([[38.91529142, -76.98528226, 'Ward 5'],
       [38.90944603, -77.04564824, 'Ward 2'],
       [38.8660913, -76.98214002, 'Ward 8'],
       [38.92341947, -77.04313318, 'Ward 1'],
       [38.9463559, -77.03300165, 'Ward 4'],
       [38.91340013, -76.98829046, 'Ward 5'],
       [38.94610649, -77.09612693, 'Ward 3'],
       [38.90541497, -77.04990657, 'Ward 2'],
       [38.90488356, -77.05750979, 'Ward 2'],
       [38.92097326, -77.07227167, 'Ward 3'],
       [38.90615368, -77.04205271, 'Ward 2'],
       [38.92092683, -77.07161294, 'Ward 3'],
       [38.90988803, -77.06445465, 'Ward 2'],
       [38.89592455, -77.02216168, 'Ward 2'],
       [38.91029431, -76.99642067, 'Ward 5'],
       [38.91339811, -77.04602055, 'Ward 2'],
       [38.90764641, -77.04193, 'Ward 2'],
       [38.91162884, -77.04518543, 'Ward 2'],
       [38.92543543, -77.10231797, 'Ward 3'],
       [38.89490228, -76.91339924, 'Ward 7']], dtype=object)

In [6]:
df[["LATITUDE","LONGITUDE",]][0:10].to_numpy()

array([[ 38.94611252, -77.09543099],
       [ 38.88943338, -77.00276751],
       [ 38.90117869, -77.04405503],
       [ 38.90499081, -77.06131388],
       [ 38.92939756, -77.02359941],
       [ 38.9043907 , -77.06245403],
       [ 38.9005195 , -77.03066994],
       [ 38.9451418 , -77.06368319],
       [ 38.91574888, -77.03738369],
       [ 38.92940307, -77.03729582]])

In [7]:
df[["X","Y","LATITUDE","LONGITUDE"]]

,X,Y,LATITUDE,LONGITUDE
0,391727.30,142028.47,38.946113,-77.095431
1,399759.90,135732.29,38.889433,-77.002768
2,396178.56,137037.03,38.901179,-77.044055
3,394681.77,137461.07,38.904991,-77.061314
4,397953.74,140168.94,38.929398,-77.023599
...,...,...,...,...
2125,399037.23,136551.16,38.896810,-77.011099
2126,396370.32,138598.40,38.915245,-77.041853
2127,394750.17,137509.83,38.905430,-77.060526
2128,395634.40,137367.41,38.904152,-77.050330


In [8]:
# GEOM STUFF

# Number of kilometers in one radian
kms_per_radian = 6371.0088

# represent points consistently as (lat, lon)
coords = df[['LATITUDE', 'LONGITUDE']].values

# Convert list of lat & lon to radians because scikit learn harversine needs radian
X = np.radians(coords)

In [9]:
min_samples = 10
epsilon = 0.5


db1 = cluster.DBSCAN(eps=epsilon/kms_per_radian,
                     min_samples=min_samples,
                     algorithm='ball_tree',
                     metric='haversine').fit(X)

In [10]:
db1.labels_.min()

-1

In [11]:
min_samples = 5
epsilon = 0.2

db2 = cluster.DBSCAN(eps=epsilon/kms_per_radian,
                     min_samples=min_samples,
                     algorithm='ball_tree',
                     metric='haversine').fit(X)

In [12]:
db2.labels_.min()

-1

In [13]:
min_cluster_size = 15
min_samples = 3

hdb1 = hdbscan.HDBSCAN(algorithm = 'best',
                       min_cluster_size=min_cluster_size,
                       min_samples=min_samples,
                       metric='haversine').fit(coords)

In [14]:
hdb1.labels_.min()

-1

In [15]:
min_cluster_size = 5
min_samples = 2

hdb2 = hdbscan.HDBSCAN(algorithm = 'best',
                       min_cluster_size=min_cluster_size,
                       min_samples=min_samples,
                       metric='haversine').fit(coords)

In [16]:
hdb2.labels_.min()

-1

In [17]:
min_cluster_size = 10
min_samples = 5
epsilon = 1
sel_method = 'eom'

hybrid1 = hdbscan.HDBSCAN(algorithm = 'best',
                          min_cluster_size=min_cluster_size,
                          min_samples=min_samples,
                          metric='haversine',
                          cluster_selection_epsilon = epsilon/kms_per_radian,
                          cluster_selection_method = sel_method).fit(coords)

In [18]:
hybrid1.labels_.min()

-1

In [19]:
min_cluster_size = 10
min_samples = 1
epsilon = 5
sel_method = 'leaf'

hybrid2 = hdbscan.HDBSCAN(algorithm = 'best',
                          min_cluster_size=min_cluster_size,
                          min_samples=min_samples,
                          metric='haversine',
                          cluster_selection_epsilon = epsilon/kms_per_radian,
                          cluster_selection_method = sel_method).fit(coords)

In [20]:
hybrid2.labels_.min()

-1

In [21]:
# Define a color domain for d3
def color_domain(column):
    col_min = column.min()
    col_max = column.max()
    return col_min, col_max

In [22]:
color_domain(db1.labels_)

(-1, 10)

In [23]:
color_domain(db2.labels_)

(-1, 55)

In [27]:
domain_min, domain_max = [list(x) for x in zip(color_domain(db1.labels_), color_domain(db2.labels_),
                               color_domain(hdb1.labels_), color_domain(hdb2.labels_),
                               color_domain(hybrid1.labels_), color_domain(hybrid2.labels_))]

In [28]:
domain_min

[-1, -1, -1, -1, -1, -1]

In [29]:
domain_max

[10, 55, 47, 198, 65, 55]

In [30]:
df['labels'] = [list(x) for x in zip(db1.labels_, db2.labels_,hdb1.labels_,hdb2.labels_,hybrid1.labels_,hybrid2.labels_)]

In [31]:
df['domain_min'] = [domain_min for i in df.index]
df['domain_max'] = [domain_max for i in df.index]

In [32]:
df = df[['OBJECTID','LONGITUDE','LATITUDE','TRADE_NAME','ADDRESS',
         'CLASS','TYPE', 'WARD','ZIPCODE','TOTAL_CAPACITY','labels', 'domain_min', 'domain_max']]

In [33]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(x=df.LONGITUDE, y=df.LATITUDE)
)



In [34]:
gdf.dtypes

OBJECTID             int64
LONGITUDE          float64
LATITUDE           float64
TRADE_NAME          object
ADDRESS             object
CLASS               object
TYPE                object
WARD                object
ZIPCODE              int64
TOTAL_CAPACITY     float64
labels              object
domain_min          object
domain_max          object
geometry          geometry
dtype: object

In [35]:
# Because JSON doesn't like numpy arrays, convert them to pandas int
# https://github.com/automl/SMAC3/issues/453


def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()

In [36]:
# Cannot use geopandas to_file because each row of labels is a list 
# https://issueexplorer.com/issue/geopandas/geopandas/2113

with open("../GeoJSON/labeled.geojson", "w") as f:
    json.dump(gdf.__geo_interface__, f, default=myconverter)

In [37]:
with open("../GeoJSON/labeled.geojson", "r") as f:
    features = json.load(f)
    gdf2 = gpd.GeoDataFrame.from_features(features)

In [38]:
gdf2

,geometry,ADDRESS,CLASS,LATITUDE,LONGITUDE,OBJECTID,TOTAL_CAPACITY,TRADE_NAME,TYPE,WARD,ZIPCODE,domain_max,domain_min,labels
0,POINT (-77.09543 38.94611),4822 YUMA ST NW,C,38.946113,-77.095431,65,199.0,Decarlos Restaurant,Restaurant,Ward 3,20016,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[-1, 0, -1, 9, -1, -1]"
1,POINT (-77.00277 38.88943),201 EAST CAPITOL ST SE,C,38.889433,-77.002768,66,NaN,Folger Shakespeare Library,Multipurpose,Ward 6,20003,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[0, -1, -1, -1, -1, -1]"
2,POINT (-77.04406 38.90118),1912 I ST NW,C,38.901179,-77.044055,67,96.0,Chalin's Restaurant,Restaurant,Ward 2,20006,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[0, 1, 39, 190, 60, 50]"
3,POINT (-77.06131 38.90499),3100 M ST NW,A,38.904991,-77.061314,68,NaN,Potomac Wines And Spirits,Retail - Liquor Store,Ward 2,20007,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[0, 2, 34, 150, 40, 46]"
4,POINT (-77.02360 38.92940),3108 GEORGIA AVE NW,B,38.929398,-77.023599,69,NaN,Kusa Market,Retail - Grocery,Ward 1,20010,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[0, 3, 21, 83, 31, 33]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,POINT (-77.01110 38.89681),525 NEW JERSEY AVE NW,C,38.896810,-77.011099,2126,14.0,Hilton Washington DC Capitol Hill,Hotel,Ward 6,20001,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[0, 27, 27, 89, 35, 39]"
2119,POINT (-77.04185 38.91524),1836 18TH ST NW,C,38.915245,-77.041853,2127,96.0,TBD,Restaurant,Ward 2,20009,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[0, 7, 32, 113, 41, 47]"
2120,POINT (-77.06053 38.90543),3057 M ST NW,C,38.905430,-77.060526,2128,80.0,Amigo Mio,Restaurant,Ward 2,20007,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[0, 2, 34, -1, 40, 46]"
2121,POINT (-77.05033 38.90415),1118 23rd ST NW,C,38.904152,-77.050330,2129,708.0,Imperfecto,Restaurant,Ward 2,20037,"[10, 55, 47, 198, 65, 55]","[-1, -1, -1, -1, -1, -1]","[0, 1, 44, 170, 59, 50]"
